In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
head = '../input/another-animals-dataset/ex_imgs/'

cat_list = list(sorted(os.listdir(head)))
cat_list

In [ ]:
class_csv = pd.read_csv('../input/another-animals-dataset/labels/classes.txt', sep='\t', header=None)
class_csv.sample(5)

In [ ]:
class_to_label_dict = {}
for cls in class_csv.values:
    class_to_label_dict[cls[1]] = cls[0] - 1

label_to_class_dict = {v: k for k, v in class_to_label_dict.items()}

In [ ]:
label_to_class_dict[0], class_to_label_dict[label_to_class_dict[0]]

In [ ]:
cat_ = ['bobcat']

In [ ]:
img_list = []
label_list = []


for cat in cat_:
    c_img_list = [head + cat + '/' + ipl for ipl in os.listdir(head + cat)]
    img_list += c_img_list
    label_list += [class_to_label_dict[cat]] * len(c_img_list)
    
len(img_list), img_list[0], len(label_list), label_list[0]

In [ ]:
a_head = '../input/awa-dataset/imgs_e/'

In [ ]:
for cat in cat_:
    c_img_list = [a_head + cat + '/' + ipl for ipl in os.listdir(a_head + cat)]
    img_list += c_img_list
    label_list += [class_to_label_dict[cat]] * len(c_img_list)
    
len(img_list), img_list[-1], len(label_list), label_list[-1]

### try imgs...

In [ ]:
from PIL import Image, ImageOps, ImageEnhance

import matplotlib.pyplot as plt

img = Image.open(img_list[0])
plt.imshow(img)
plt.show()

In [ ]:
img = Image.open(img_list[-1])
plt.imshow(img)
plt.show()

In [ ]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models
import torchvision.transforms as transforms

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
class AwaDataSet(Dataset):
    #  img_path_list: text. labels: text, need to locate through
    def __init__(self, img_path_list, x_transform):
        self.img_path_list = img_path_list
        self.x_transform = x_transform

    def __getitem__(self, idx):
        
        try:
            img = Image.open(self.img_path_list[idx])
        except:
            img = Image.fromarray(np.random.randint(0, 255, (64, 64, 3), dtype=np.uint8))
                
        if head in self.img_path_list[idx]:
            if np.random.randint(1000) <= 125:
                img = ImageOps.mirror(img)

            if np.random.randint(1000) <= 50:
                img = img.rotate(np.random.randint(-18, 18))

        else:
            img = img.convert('RGBA')
            
            blank = img.copy()
        
            if np.random.randint(1000) <= 125:
                img = ImageOps.mirror(img)

            if np.random.randint(1000) <= 50:
                img = img.rotate(np.random.randint(-18, 18), expand=True)
                
            width, height = img.size
            target_size = max(img.size)

            blank = blank.resize((target_size * 3, target_size * 3))
            blank = blank.crop((target_size, target_size, 2 * target_size, 2 * target_size))
            
            blank.paste(img, (int((max(img.size) - width) / np.random.uniform(1.75, 2.25)),
                          int((max(img.size) - height) / np.random.uniform(1.75, 2.25))), img)
            img = blank
            
        img = img.convert('RGB')
        img = img.resize((128, 128))
        img = np.array(img, dtype=np.float32) / 255

        if np.random.randint(1000) <= 125:
            img = np.clip(img * np.random.uniform(0.8, 1.28), 0, 1)
        
        img = self.x_transform(img)
        
#         label = self.label_list[idx]

        return img

    def __len__(self):
        return len(self.img_path_list)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,) * 3, std=(0.5,) * 3),
])

In [ ]:
train_dataset = AwaDataSet(img_list, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
for db in train_loader:
    img = db
#     print(img.mean())
#     print(label)
#     print(label)

    img = img[0].cpu().data.numpy()
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img * 0.5 + 0.5)
    plt.show()

    break

### weight initialized function

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### crazy idea... again...

### Define the auto_encoder(this one differs to the vae one, this one contains only the encoder part, and the decoder also works as the generator...)

In [ ]:
class auto_encoder(nn.Module):
    def __init__(self):
        super(auto_encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 64 x 64
            
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.InstanceNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 32 x 32
            
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 16 x 16 
            
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5),
            # size: 8 x 8
            
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5),
            # size: 4 x 4
            
            nn.Conv2d(512, 1024, 4, 2, 1),
            nn.InstanceNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5),
            # size: 2 x 2
            
            nn.Conv2d(1024, 2048, 4, 2, 1),
            # size: 1 x 1
        )
        
        self.mu = nn.Linear(2048, 256)
        self.logvar = nn.Linear(2048, 256)
            
    def encode(self, x):
        x = self.encoder(x)
        x = x.view(-1, 2048)
        
        return self.mu(x), self.logvar(x)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        
        return mu + eps * std
        
    def forward(self, x):
        x = x.view(-1, 3, 128, 128)
        mu, logvar = self.encode(x)
        x = self.reparameterize(mu, logvar)
#         x = self.encoder(x)
        
        x = (x - x.mean(dim=1, keepdim=True)) / (x.std(dim=1, keepdim=True) + 1e-8)
        x = x.view(-1, 256, 1, 1)        
                
        return x, mu, logvar

In [ ]:
AE = auto_encoder().cuda()
summary(AE, (3, 128, 128))

In [ ]:
AE.apply(weights_init)
print('init...')

### Loss Function

In [ ]:
def criterion_vae(rec_imgs, imgs, mu, logvar):
    
    ### remember: the range of output image is (-1, 1),
    ### without making it to range(0, 1), using bce_loss would raise some strange error.
    
#     print(rec_imgs.max(), rec_imgs.min(), imgs.max(), imgs.min())
    bce_loss = F.binary_cross_entropy(rec_imgs.view(-1, 3 * 128 * 128), imgs.view(-1, 3 * 128 * 128), reduction='sum')
    
    kld_loss = -0.5 * torch.sum(1 + logvar - mu ** 2 - logvar.exp())
    
    return (bce_loss + kld_loss) / rec_imgs.size()[0]

In [ ]:
from torch.nn import Parameter

def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)


class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False


    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)


    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)

### Define self attention layer

In [ ]:
class Self_Attn(nn.Module):
    def __init__(self, in_dim):
        super(Self_Attn, self).__init__()
        self.channel_in = in_dim
        
        self.Q_conv = nn.Conv2d(in_dim, in_dim//8, 1, 1, 0)
        self.K_conv = nn.Conv2d(in_dim, in_dim//8, 1, 1, 0)
        self.V_conv = nn.Conv2d(in_dim, in_dim, 1, 1, 0)
        
        self.gamma = nn.Parameter(torch.zeros(1))
        
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        b_size, c, width, height = x.size()
        
        proj_Q = self.Q_conv(x).view(b_size, -1, width * height).permute(0, 2, 1)
        proj_K = self.K_conv(x).view(b_size, -1, width * height)
        
        energy = torch.bmm(proj_Q, proj_K)
        attn = self.softmax(energy)
        
        proj_V = self.V_conv(x).view(b_size, -1, width * height)
        
        out = torch.bmm(proj_V, attn.permute(0, 2, 1))
        out = out.view(b_size, c, width, height)
        
        out = self.gamma * out + x
        
        return out

### Define Generator

In [ ]:
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        
        self.main = nn.Sequential(
            SpectralNorm(nn.ConvTranspose2d(512, 1024, 4, 1, 0, bias=False)),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 4 x 4

            SpectralNorm(nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 8 x 8

            SpectralNorm(nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            # size: 16 x 16

            SpectralNorm(nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
#             Self_Attn(128),
            nn.Dropout(0.25),
            # size: 32 x 32
            
            SpectralNorm(nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
#             Self_Attn(64),
            nn.Dropout(0.25),
            # size: 64 x 64     
            
#             nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
#             nn.Tanh()
            # size: 128 x 128 
        )
        
        self.gen_output = nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False)
        self.ae_output = nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x, mode='gen'):  
        
        x = self.main(x)
        
        if mode == 'gen':
            x = self.gen_output(x)
            x = self.tanh(x)
            
        elif mode == 'ae':
            x = self.ae_output(x)
            x = self.sigmoid(x)
        
        return x

In [ ]:
G = generator().cuda()
summary(G, [(512, 1, 1)])

In [ ]:
G.apply(weights_init)
print('init...')

### define discriminator

In [ ]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.main = nn.Sequential(
            SpectralNorm(nn.Conv2d(3, 32, 4, 2, 1, bias=False)),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 64 x 64
            
            SpectralNorm(nn.Conv2d(32, 64, 4, 2, 1, bias=False)),
#             nn.BatchNorm2d(64),
            nn.InstanceNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 32 x 32
            
            SpectralNorm(nn.Conv2d(64, 128, 4, 2, 1, bias=False)),
#             nn.BatchNorm2d(128),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # size: 16 x 16
            
            SpectralNorm(nn.Conv2d(128, 256, 4, 2, 1, bias=False)),
#             nn.BatchNorm2d(256),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            # size: 8 x 8
            
            SpectralNorm(nn.Conv2d(256, 512, 4, 2, 1, bias=False)),
#             nn.BatchNorm2d(512),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
#             Self_Attn(512),
            # size: 4 x 4
            
            SpectralNorm(nn.Conv2d(512, 1024, 4, 2, 1, bias=False)),
#             nn.BatchNorm2d(1024),
            nn.InstanceNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
#             Self_Attn(1024),
            # size: 2 x 2
            
#             nn.Conv2d(1024, 50 + 1, 4, 2, 1, bias=False),
            nn.Conv2d(1024, 1, 4, 2, 1, bias=False),

#             nn.Sigmoid(),
        )
                
    def forward(self, x):
        x = self.main(x)
#         x = x.view(-1, 50 + 1)
        x = x.view(-1)
        
        return x

In [ ]:
D = discriminator().cuda()
summary(D, (3, 128, 128))

In [ ]:
D.apply(weights_init)
print('init...')

### gp loss

In [ ]:
from torch import autograd
def cal_gp(d_model, real_imgs, fake_imgs):
    
    b_size = real_imgs.size()[0]
    
    alpha = torch.rand(b_size, 1, 1, 1)
    alpha = alpha.expand_as(real_imgs).cuda()
    
    inter = alpha * real_imgs + (1 - alpha) * fake_imgs
    inter = inter.cuda()
    inter = autograd.Variable(inter, requires_grad=True)
    
    disc = d_model(inter)
    
    grads = autograd.grad(outputs=disc, inputs=inter,
                         grad_outputs=torch.ones_like(disc).cuda(),
                         create_graph=True, retain_graph=True, only_inputs=True)[0]
    grads = grads.view(grads.size(0), -1)
    
    gp = ((grads.norm(2, dim=1) - 1) ** 2).mean()
    return gp

In [ ]:
criterion_bce = nn.BCELoss()
criterion_mse = nn.MSELoss()
criterion_ce = nn.CrossEntropyLoss()

g_optimizer = optim.Adam(G.parameters(), lr=5e-5, betas=(0.5, 0.9))
d_optimizer = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5, 0.9))
# g_optimizer = optim.RMSprop(G.parameters(), lr=1e-5)
# d_optimizer = optim.RMSprop(D.parameters(), lr=1e-5)

ae_optimizer = optim.Adam(AE.parameters(), lr=1e-4, betas=(0.5, 0.9))
ag_optimizer = optim.Adam(G.parameters(), lr=1e-4, betas=(0.5, 0.9))

# ae_scheduler = lr_scheduler.ExponentialLR(ae_optimizer, gamma=0.999)
# ag_scheduler = lr_scheduler.ExponentialLR(ag_optimizer, gamma=0.999)


In [ ]:
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 6.4, 6.4

In [ ]:
d_epoch = 1
g_epoch = 1
ae_epoch = 1

In [ ]:
nc = 2e-1
decay = 0.9995

for epoch in range(1000):
    
    d_running_loss = 0.0
    g_running_loss = 0.0
    ae_running_loss = 0.0
    
    if epoch % 10 == 9 or epoch == 0:
        print('nc:', nc)
    
    for i, data_batch in enumerate(train_loader):
        real_imgs = data_batch
        real_imgs = real_imgs.cuda()
        
        b_size = real_imgs.size()[0]
        
        labels_real = torch.ones(b_size).cuda()
        labels_fake = torch.zeros(b_size).cuda()
        labels_real_ = torch.ones(b_size).cuda()
        
        labels_real_s = 0.7 * torch.ones(b_size).cuda() + 0.55 * torch.rand(b_size).cuda()
        labels_fake_s = 0.25 * torch.rand(b_size).cuda()
        
        if np.random.randint(1000) < 100:
            labels_real, labels_fake = labels_fake, labels_real
            labels_real_s, labels_fake_s = labels_fake_s, labels_real_s
       
        for _ in range(ae_epoch):
            AE.zero_grad()
            G.zero_grad()

#             imgs_enc, mu, logvar = AE(real_imgs)
            imgs_enc, mu, logvar = AE(real_imgs)
            z = torch.randn(b_size, 256, 1, 1).cuda()
            imgs_enc_ = torch.cat((z, imgs_enc), dim=1)
        
            rec_imgs = G(imgs_enc_, mode='ae')
    #         loss = criterion(rec_imgs.view(b_size, -1) * 0.5 + 0.5, real_imgs.view(b_size, -1) * 0.5 + 0.5)
            ae_loss = 1e-2 * criterion_vae(rec_imgs, real_imgs * 0.5 + 0.5, mu, logvar)
#             ae_loss = criterion_bce(rec_imgs, real_imgs * 0.5 + 0.5)

            ae_loss.backward()

            ae_optimizer.step()
            ag_optimizer.step()

            ae_running_loss += ae_loss.item()
                        
        for _ in range(d_epoch):
            D.zero_grad()
            
            d_real_outputs = D(real_imgs + nc * torch.randn_like(real_imgs).cuda())
#             d_real_loss = criterion_mse(d_real_outputs, labels_real_s)
            d_real_loss = d_real_outputs.view(-1).mean()
            
            z = torch.randn(b_size, 512, 1, 1).cuda()
#             fake_enc = RE(z).detach()
            fake_imgs = G(z, mode='gen').detach()
            
            d_fake_outputs = D(fake_imgs + nc * torch.randn_like(fake_imgs).cuda())
#             d_fake_loss = criterion_mse(d_fake_outputs, labels_fake_s)
            d_fake_loss = d_fake_outputs.view(-1).mean()
    
            d_gp_loss = cal_gp(D, real_imgs, fake_imgs)
            
            d_loss = d_fake_loss - d_real_loss + 10 * d_gp_loss
            
#             d_loss = d_real_loss + d_fake_loss
            d_loss.backward(retain_graph=True)
            
            d_optimizer.step()
            
            d_running_loss += d_loss.item()
            
            nc *= decay
            
        for _ in range(g_epoch):
            G.zero_grad()
            
            z = torch.randn(b_size, 512, 1, 1).cuda()
            fake_imgs = G(z, mode='gen')
            
            d_outputs = D(fake_imgs)
            
#             g_loss = criterion_mse(d_outputs, labels_real_)
            g_loss = - d_outputs.view(-1).mean()
            g_loss.backward()
            
            g_optimizer.step()
            
            g_running_loss += g_loss.item()
        
        t = 30
        if (epoch % 10 == 9 or epoch == 0) and i == 0:
#         if i % t == t-1 or i == 0:

            print(epoch+1, (i+1) * 32, 
                    'd_running_loss:', d_running_loss/ (t if i != 0 else 1),
                    'g_running_loss:', g_running_loss/ (t if i != 0 else 1),
                 )
            print('ae_running_loss:', ae_running_loss/ (t if i != 0 else 1))
                        
            d_running_loss = 0.0
            g_running_loss = 0.0
            ae_running_loss = 0.0
            
            real_samples = real_imgs[:4]
            real_samples_ = real_samples.cpu().data.numpy()
            real_samples_ = np.transpose(real_samples_, (0, 2, 3, 1))            
            
            plt.subplots_adjust(wspace=0.025, hspace=0.025)
            plt.grid(False)
            for k in range(4):
                plt.subplot(1, 4, k+1)
#                 plt.title(label_to_class_dict[f_labels[k]])
                plt.axis('off')

                plt.imshow(real_samples_[k] * 0.5 + 0.5)
            plt.show()
            
            enc_samples, _, _ = AE(real_samples)
            z = torch.randn(4, 256, 1, 1).cuda()
            enc_samples_ = torch.cat((z, enc_samples), dim=1)
            rec_samples = G(enc_samples_, mode='ae')
            rec_samples_ = rec_samples.cpu().data.numpy()
            rec_samples_ = np.transpose(rec_samples_, (0, 2, 3, 1))
            
            plt.subplots_adjust(wspace=0.025, hspace=0.025)
            plt.grid(False)
            for k in range(4):
                plt.subplot(1, 4, k+1)
#                 plt.title(label_to_class_dict[f_labels[k]])
                plt.axis('off')

                plt.imshow(rec_samples_[k])
            plt.show()
            
            z = torch.randn(4, 512, 1, 1).cuda()
            fake_samples = G(z).cpu().data.numpy()
            fake_samples = np.transpose(fake_samples, (0, 2, 3, 1))
            
            plt.subplots_adjust(wspace=0.025, hspace=0.025)
            plt.grid(False)
            for k in range(4):
                plt.subplot(1, 4, k+1)
#                 plt.title(label_to_class_dict[f_labels[k]])
                plt.axis('off')

                plt.imshow(fake_samples[k] * 0.5 + 0.5)
            plt.show()
            
            

#         break
#     break